In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Open the text file in read mode
with open('/content/drive/MyDrive/next_word_pred.txt', 'r') as file:
    # Read the entire contents of the file into a variable
    file_data = file.read()

# Print the data read from the file
print(file_data)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts([file_data])

In [ ]:
len(tokenizer.word_index)

In [ ]:
input_sequences = []
for sentence in file_data.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [ ]:
input_sequences

In [ ]:
max_len = max([len(x) for x in input_sequences])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [ ]:
padded_input_sequences

In [ ]:
X = padded_input_sequences[:,:-1]

In [ ]:
y = padded_input_sequences[:,-1]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=8932)

In [ ]:
y.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential()
model.add(Embedding(8932, 128, input_length=19))
model.add(LSTM(300))
# model.add(LSTM(150))
model.add(Dense(8932, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X,y,epochs=50, batch_size = 32, validation_split = 0.2)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.15, patience=2, min_lr=0.0001)

In [ ]:
model.save('NWP.h5')

In [ ]:
import time
import numpy as np
text = "The movie"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=19, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

In [ ]:
tokenizer.word_index